In [16]:
import pandas as pd
import numpy as np

cwd = 'C:/Users/klouc/Desktop/slovcho/spell_checking'

sents = pd.read_csv(rf"{cwd}/sents.csv",encoding='utf-8')
valid_words = pd.read_csv(rf"{cwd}/single_words_bg.csv",encoding='utf-8')

valid_words_dict = {}
for word in list(valid_words['word']):
    valid_words_dict[word] = word

print("Number of sentences:")
print(len(sents.values))
print("Number of words:")
print(len(valid_words.values))

Number of sentences:
1758815
Number of words:
736456


In [17]:
# TODO: Tokenise input in clever way
# TODO: Dealing with non-cyrillic characters
# TODO: How to deal if first word (capitalised but only cause start of sent) is wrong? As in, discerning capital letter

import re

sent_to_check = 'Възстанието беше тогава'

# PRE-PROCESS INPUT

# Clear punctuation
sent_to_check = sent_to_check
sent_to_check = re.sub("[?!.,:]", "", sent_to_check)

# Tokenise
words_to_check = sent_to_check.split(' ')

# Lower first character
if(ord(words_to_check[0][0]) >= ord('А') and ord(words_to_check[0][0]) <= ord('Я')):
     words_to_check[0] = words_to_check[0].lower()
# # If beginning word is capitalised and a non-capitalised version exists in the dict, lower it (might be a name)
# # Else do not lower
# if(ord(words_to_check[0][0]) >= ord('А') and ord(words_to_check[0][0]) <= ord('Я')):
#     if(words_to_check[0].lower() in valid_words.values):
#         words_to_check[0] = words_to_check[0].lower()

In [22]:
# TODO: Break sentences up more as some 'sents' include multiple sentences
# TODO: Lowers first letter of sent but what if first word has multiple capitalised letters
# TODO: Tokenise sentences in a more clever way

# Create unigrams and bigrams from sents

# Sents vocab АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЬЮЯабвгдежзийклмнопрстуфхцчшщъьюя1234567890!@#$%^&*()-_=+`~[]{}|;':"\,./<>
def process_sent(sentence):

    replace_with_space_chars = '[!@#$%^&*()-_=+`~{}\|;\[\]\':\"\,./<>]'

    # CLEAN SENTENCES
    # Replace some chars with space
    sentence = re.sub(replace_with_space_chars, ' ', sentence)
    # Replace consecutive spaces with single space
    sentence = re.sub(r'\s+', ' ', sentence)

    # If sent below 10 characters do not consider
    if(len(sentence) < 5): return np.nan

    # Remove first charatcer if space
    if(sentence[0] == ' '): sentence = sentence[1:]
    # Lower letter if non-capitlised word exists in dict
    first_word = ''
    if(sentence.find(' ')):
        first_word = sentence[:sentence.find(' ')]
    else:
        first_word = sentence
    if(ord(first_word[0]) >= ord('А') and ord(first_word[0]) <= ord('Я') and (first_word[0].lower() + first_word[1:]) in valid_words_dict):
        sentence = sentence[0].lower() + sentence[1:]

    # ADD BEGINNING AND CLOSING TAG
    sentence = '<s>' + sentence + '</s>'

    return sentence
    
cleaned_sents = sents["sent"].apply(lambda x : process_sent(x))
# Drop rows with no content
cleaned_sents.dropna(inplace=True)
cleaned_sents.reset_index(inplace=True, drop=True)

cleaned_sents.head(10)

1758815
1758763


In [19]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'абвгдежзийклмнопрстуфхцчшщъьюя'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

# Find non-real words and evaluate
for word in words_to_check:
    if word not in valid_words.values:
        candidates = edits1(word)

        # Get candidates which are valid words
        valid_candidates = []
        for candidate in candidates:
            if(candidate in valid_words.values):
                valid_candidates.append(candidate)
        print(valid_candidates)


KeyboardInterrupt: 